In [ ]:
import ufl
import numpy as np
import time
from dolfinx import fem, mesh, io
from mpi4py import MPI
from petsc4py import PETSc
from matplotlib import pyplot as plt
# from util import *
from elasticity.material import Material_el_lin_iso  
from input_files import *
from input_files.Cantilever import Cantilever2D
from elasticity.material import *
from dolfinx import mesh, fem

prob = Cantilever2D()
mat = Material_el_lin_iso()


lmd, mu = mat.lmd, mat.mu

def epsilon(_u):
            return ufl.sym(ufl.grad(_u))
def sigma(_u, lmd, mu):
    return 2.0 * mu * epsilon(_u) + lmd * ufl.tr(epsilon(_u)) * ufl.Identity(len(_u))

U1 = fem.VectorFunctionSpace(prob.msh, ("CG", 1)) # displacement basiss
D0 = fem.FunctionSpace(prob.msh, ("DG", 0)) # density
u, v = ufl.TrialFunction(U1), ufl.TestFunction(U1) # note that this is ufl
u_sol = fem.Function(U1) 

k = ufl.inner(sigma(u, lmd, mu), ufl.grad(v)) * ufl.dx

